# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,loreto,22.2667,-101.9667,70.72,47,96,6.91,MX,1689867454
1,1,tari,12.4086,4.4914,96.87,37,32,6.80,NG,1689867454
2,2,edinburgh of the seven seas,-37.0676,-12.3116,53.29,64,34,24.90,SH,1689867454
3,3,qaqortoq,60.7167,-46.0333,47.53,87,100,7.90,GL,1689867347
4,4,waslala,13.2333,-85.3833,83.61,68,37,8.01,NI,1689867454


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot

map_plot_1= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)


# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_narrow=city_data_df[(city_data_df['Max Temp'] < 80.6) &
                             (city_data_df['Max Temp'] > 68.8) &
                             (city_data_df['Wind Speed'] < 6) &
                             (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
city_data_narrow=city_data_narrow.dropna() # drop any rows will null values

# Display sample data
city_data_narrow.count()

City_ID       7
City          7
Lat           7
Lng           7
Max Temp      7
Humidity      7
Cloudiness    7
Wind Speed    7
Country       7
Date          7
dtype: int64

In [5]:
city_data_narrow

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,51,the pas,53.8251,-101.2541,71.62,60,0,4.61,CA,1689867460
134,134,chilliwack,49.1747,-121.9443,80.17,63,0,1.14,CA,1689867469
159,159,jackson,42.4165,-122.8345,73.00,61,0,1.01,US,1689867472
213,213,rupert,42.6191,-113.6772,71.55,56,0,5.75,US,1689867205
216,216,newman,37.3138,-121.0208,75.54,24,0,5.55,US,1689867360
238,238,waingapu,-9.6567,120.2641,71.76,78,0,5.77,ID,1689867487
253,253,trinidad,-14.8333,-64.9000,76.98,50,0,0.00,BO,1689867490


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= city_data_narrow[["City", "Lat", "Lng", "Country", "Humidity"]].copy()
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Lat,Lng,Country,Humidity,Hotel Name
51,the pas,53.8251,-101.2541,CA,60,
134,chilliwack,49.1747,-121.9443,CA,63,
159,jackson,42.4165,-122.8345,US,61,
213,rupert,42.6191,-113.6772,US,56,
216,newman,37.3138,-121.0208,US,24,
238,waingapu,-9.6567,120.2641,ID,78,
253,trinidad,-14.8333,-64.9000,BO,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": geoapify_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
# Display sample data
hotel_df

Retrieving Results for Index 51: the pas.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
------------
Retrieving Results for Index 134: chilliwack.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
------------
Retrieving Results for Index 159: jackson.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
------------
Retrieving Results for Index 213: rupert.
{
    "error_message": "The provided API key is invalid.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
------------
Retrieving Results for Index 216: newman.
{
    "error_message": "The provided 

,City,Lat,Lng,Country,Humidity,Hotel Name
51,the pas,53.8251,-101.2541,CA,60,
134,chilliwack,49.1747,-121.9443,CA,63,
159,jackson,42.4165,-122.8345,US,61,
213,rupert,42.6191,-113.6772,US,56,
216,newman,37.3138,-121.0208,US,24,
238,waingapu,-9.6567,120.2641,ID,78,
253,trinidad,-14.8333,-64.9000,BO,50,


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE